In [33]:

from kafka import KafkaProducer, KafkaConsumer
from time import sleep
import json
from datetime import datetime
from confluent_kafka import Producer
from pymongo import MongoClient, ASCENDING, DESCENDING
from pprint import pprint
from itertools import combinations, combinations_with_replacement
from kafka.admin import KafkaAdminClient, NewTopic
from EtudeBinance import getpairs

# Définition des paramètres

**Producer** permet d'envoyer des données tandis que **consumer** permet de lire en direct.

Le client de mongo permet de stocker ces valeurs dans notre database.

In [34]:
producerconnect = Producer({'bootstrap.servers': 'localhost:9092'})
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         api_version=(0,10,2))
consumer = KafkaConsumer(
    'ApiBinance',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     auto_commit_interval_ms=300,
     enable_auto_commit=True,
     group_id='my-group')

client = MongoClient("localhost", 27017)
db = client.Binance
collection = db.symbol
db.symbol.delete_many({})

**A CHAQUE LANCEMENT DU PROGRAMME MENU ON SUPPRIME ET RECREE LE TOPIC POUR REINITIALISER**

In [35]:
def SupprimeTOPIC():
    admin_client = KafkaAdminClient(bootstrap_servers=['localhost:9092'])
    topic_names = ['ApiBinance']
    def create_topics(topic_names):

        existing_topic_list = consumer2.topics()
        print(list(consumer2.topics()))
        topic_list = []
        for topic in topic_names:
            if topic not in existing_topic_list:
                print('Topic : {} added '.format(topic))
                topic_list.append(NewTopic(name=topic))
                #topic_list.append(NewTopic(name=topic, num_partitions=3, replication_factor=3))
            else:
                print('Topic : {} already exist '.format(topic))
        try:
            if topic_list:
                admin_client.create_topics(new_topics=topic_list, validate_only=False)
                print("Topic Created Successfully")
            else:
                print("Topic Exist")
                delete_topics(topic_names)
        except Exception as e:
            print(e)

    def delete_topics(topic_names):
        try:
            admin_client.delete_topics(topics=topic_names)
            print("Topic Deleted Successfully")
            create_topics(topic_names)
        except Exception as e:
            print(e)


    consumer2 = KafkaConsumer(
        bootstrap_servers = ['localhost:9092'],
        )
    create_topics(topic_names)

# Insertion

On a récupérer les données du fichier. On utilise Kafka Connect pour faire le lien entre mongodb et les topics.

On insere les données dans un topics.

In [36]:
def ApacheRemplissage():
	listeInsertion=[]
	SupprimeTOPIC()
	%run '/home/souhail/Documents/Designed Distributed Systems/EtudeBinance.py'
	with open("fichier.txt", "r") as fichier:
		
		lines = fichier.readlines()
		for line in lines:
			listeInsertion.append(line)

	list1=[]
	list1=listeInsertion[0].split(',WEEE')

	del list1[-1]

	jsone=[]
	compteurgenerale=len(list1)
	for i in list1:
		try:
			jsone.append(json.loads(i))
		except ValueError:
			print("error for",i,ValueError)

	for i in jsone:
		producer.send('ApiBinance',bytes(str(i), encoding='utf-8'))
		sleep(0.5)
	return compteurgenerale

On a inséré les données dans kafka topics. Maintenant, via Kafka connect on récupére les données puis on les met dans mongodb

**Kafka Connect**

Lire des données & on met les données dans MongoDB.

In [37]:
from confluent_kafka import Consumer, KafkaError

def conso(compteur,compteurgenerale):
    cc=1
    settings = {
        'bootstrap.servers': 'localhost:9092',
        'group.id': 'mygroup2',
        'client.id': 'client-'+str(compteur),
        'enable.auto.commit': True,
        'session.timeout.ms': 6000,
        'default.topic.config': {'auto.offset.reset': 'smallest'}
    }

    c = Consumer(settings)

    c.subscribe(['ApiBinance'])
    db.symbol.delete_many({})
    go=True
    try:
        while go:
            msg = c.poll(0.1)
            if msg is None:
                continue
            elif not msg.error():
                #print('Received message: {0}'.format(msg.value()))
                inputdonnee=collection.insert_one(json.loads(msg.value().decode('utf-8').replace("'", '"')))
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                #print("New input in our collection at "+dt_string)
                cc+=1
                if cc==compteurgenerale:
                    go=False
                    break 
            elif msg.error().code() == KafkaError._PARTITION_EOF:
                print('End of partition reached {0}/{1}'
                    .format(msg.topic(), msg.partition()))
            else:
                print('Error occured: {0}'.format(msg.error().str()))

    except KeyboardInterrupt:
        pass

    finally:
        c.close()

On actualise le topic continuellement, pour supprimer les anciennes donées grâce aux données présentes dans mongoDB.

In [38]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: {0}: {1}"
              .format(msg.value(), err.str()))
    else:
        print("Message produced: {0}".format(msg.value()))

def ProducerTerminal():
    p = Producer({'bootstrap.servers': 'localhost:9092'})
    cursor = collection.find()
    try:
        for record in cursor:
            p.produce('ApiBinance', record, callback=acked)
            p.poll(0.5)

    except KeyboardInterrupt:
        pass

    p.flush(30)
    db.symbol.delete_many({})
    conso()

On fait des requetes

In [39]:
listesymboles=["BTC","ETH","USDT","USDC","BNB","XRP","ADA","DOGE","LTC","BCH","SOL","ETC","LUNA","AVAX","SHIB","DOT","MATIC","AAVE","UNI","VRA","USDC","ADX","USDC","APE","JASMY","GALA","WAVES","LRC","EOS","TRX"]

def notation_scientifique(nombre, chiffres_apres_virgule): 
    s = str(nombre)
    exposant = len(s)
    return '%c.%se%d'%(s[0],s[1:chiffres_apres_virgule+1],exposant)
def requetesMongoDB():
    index=True
    while index:
        case=input("Do you want to read to read all the data (Press 1)" + '\n' 
        + "Read all the distinct symbol (Press 2)" + '\n' 
        + "Read the data order by the price (Press 3)" + '\n' 
        + "Read the data which are more flutuacting (Press 4)" + '\n' 
        + "Read the data order by the most traded cryptocurrencies (Press 5)" + '\n'
        + "Read and have the high price on the last 24H (Press 6)" + '\n' 
        + "Read and have the lower price on the last 24H (Press 7)" + '\n' 
        + "Read the data order by the most bought cryptocurrencies (Press 8)" + '\n' 
        +"or  exit (Press 9) > ")
        print("\n")
    
        if case == "1":
            print('Start - Read all data')
            print('\n')
            cursor = collection.find({})
            for record in cursor:
                pprint(record)
            print('\n')
            print('End - Read all data')
            print('\n')

        elif case =="2":
            print('Start - Read all the distinct symbol')
            print('\n')
            cursor = collection.distinct("symbol")
            cursor.sort()
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read all the distinct symbol')
            print('\n')

        elif case =="3":
            print('Start - Read the data order by the price - Currently ')
            print('\n')
            cursor = collection.find({"currentaverageprice":{"$gt":0}}, {"symbol":1,"currentaverageprice":1, "_id":0}).sort("currentaverageprice",DESCENDING)   
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read the data order by the price')
            print('\n')

        elif case =="4":
            print('Start - Read the data which are more flutuacting - On the last 24H ')
            print('\n')
            cursor = collection.find({"priceChangePercent" : {"$gt":0}}, {"symbol":1,"priceChangePercent":1, "_id":0}).sort("priceChangePercent",DESCENDING)   
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read the data which are more flutuacting')
            print('\n')

        elif case =="5":
            print('Start - Read the data order by the most traded cryptocurrencies - On the last 24H')
            print('\n')
            print('Information : Volume, or trading volume, is the number of units traded in a market during a given time. It is a measurement of the number of individual units of an asset that changed hands during that period.')
            print('\n')
            cursor = collection.find({"volume" : {"$gt":0}}, {"symbol":1,"volume":1, "_id":0}).sort("volume",DESCENDING)   
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read the data order by the most traded cryptocurrencies - On the last 24H')
            print('\n')
        
        elif case =="6":
            print('Start - Read and have the high price - On the last 24H ')
            print('\n')
            cursor = collection.find({"highPrice" : {"$gt":0}}, {"symbol":1,"highPrice":1, "_id":0}).sort("highPrice",DESCENDING)   
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read and have the high price - On the last 24H')
            print('\n')

        elif case =="7":
            print('Start - Read and have the lower price - On the last 24H ')
            print('\n')
            cursor = collection.find({"lowPrice" : {"$gt":0}}, {"symbol":1,"lowPrice":1, "_id":0}).sort("lowPrice",DESCENDING)   
            for record in cursor:
                print(record)
            print('\n')
            print('End - Read and have the lower price - On the last 24H')
            print('\n')

        elif case =="8":
            cursor = collection.find({"volume" : {"$gt":0}}, {"symbol":1,"volume":1, "_id":0}).sort("volume",DESCENDING) 
            listeCrypto=[]
            listeVolume=[]
            for record in cursor:
                for k,v in record.items():
                    for i in listesymboles:
                        if k=="symbol" and i in v:
                            listeCrypto.append(i)
                        if k=="volume":
                            listeVolume.append(v)
            listeFinale={}
            listeindex=[]
            for i in range(len(listeCrypto)):
                value=listeCrypto[i]
                passe=True
                for j in range(len(listeCrypto)):
                    if value == listeCrypto[j]:
                        listeindex.append(j)
                    
                
                volume=0
                for k,v in listeFinale.items():
                    if value == v:
                        passe=False
                        break
                if passe:
                    for n in listeindex:
                        volume+=listeVolume[n]
                    listeFinale[value]=volume
                listeindex=[]
            print('Start - Read the data order by the most bought cryptocurrencies (exchanged volume) - On the last 24H')
            print('\n')
            print('Information : Volume, or trading volume, is the number of units traded in a market during a given time. It is a measurement of the number of individual units of an asset that changed hands during that period.')
            print('\n')
            for k, v in sorted(listeFinale.items(), key=lambda x: x[1], reverse=True):
                print("%s: %s" % (k, v),"(On arrondit : " + notation_scientifique(v, 2) + " units)")
            print('\n')
            print('End - Read the data order by the most bought cryptocurrencies (exchanged volume) - On the last 24H')
            print('\n')

        elif case =="9":
            index=False
            print("Au revoir !")
        else:
            print("Something, you don't press an avalaible numbers, please restart")

In [40]:
print("Bonjour, bienvenue dans le menu du Projet Apache-MongoDB (Ici via l'API Binance, on traite du sujet des cryptomonnaies notamment les trades des paires de cryptomonnaies) : ")
index=True
compteur=100 #Change this value if after running this Notebook you stop it instantly (for the client-id in Apache Connect). You don't have to do this if you run and don't quit the menu (but necessary after every "run all").
while index:
    print("\n")
    print("-------------start-------------")
    print("\n")
    case=input("Do you want to read the data in the Apache topic and MongoDB (in this case, the database will be filled automatically)(Press 1) or execute some request to MongoDB (Press 2) or  exit (Press 3) > ")
    print("\n")
    if case == "1":
        compteurgenerale=ApacheRemplissage()
        conso(compteur,compteurgenerale)
        compteur+=1
    elif case =="2":
        requetesMongoDB()
    elif case =="3":
        index=False
        print("Au revoir !")
    else:
        print("Something, you don't press an avalaible numbers, please restart")
       
    print("-------------end---------------")


Bonjour, bienvenue dans le menu du Projet Apache-MongoDB (Ici via l'API Binance, on traite du sujet des cryptomonnaies notamment les trades des paires de cryptomonnaies) : 


-------------start-------------




['ApiBinance']
Topic : ApiBinance already exist 
Topic Exist
Topic Deleted Successfully
['ApiBinance']
Topic : ApiBinance added 
__init__() missing 2 required positional arguments: 'num_partitions' and 'replication_factor'
Temps d'exécution : 67.32663822174072s
-------------end---------------


-------------start-------------






Start - Read the data order by the most bought cryptocurrencies (exchanged volume) - On the last 24H


Information : Volume, or trading volume, is the number of units traded in a market during a given time. It is a measurement of the number of individual units of an asset that changed hands during that period.


USDT: 210648487108403.03 (On arrondit : 2.10e18 units)
BTC: 84272969507881.22 (On arrondit : 8.42e17 units)
USDC: 63210024524531.195 (On arro